In [45]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [46]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv 

--2020-09-06 07:25:18--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  35.5MB/s   in 2.0s   

2020-09-06 07:25:20 (35.5 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



In [47]:
df=pd.read_csv('Data-Collisions.csv')

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [49]:
# finding data types of all columns
df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [50]:
# Extracting the required columns from the dataframe i.e. we will drop the cloumns from the dataframe which are not required
# For our project, we will be dealing with predicting severity of an accident on conditions like weather conditions, road conditions and light conditions.
# According to our dataframe, the severitycode (severity) depends on the above 3 conditions, hence we will be dropping off the other columns.

df=df.drop(['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL','PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],axis=1)
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


In [51]:
#Let's find the total values for each category in a particlar column:

print(df['WEATHER'].value_counts())
print(df['ROADCOND'].value_counts())
print(df['LIGHTCOND'].value_counts())


Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64
Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64
Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, d

In [52]:
#Now since the data types for weather, Road and light conditions are object we need to use label encoding to convert the data type to numeric (int8)
#Convert data type to category first:
# Then use cat.codes:

In [53]:

df["WEATHER"] = df["WEATHER"].astype('category')
df.dtypes

df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


In [54]:
#Assign the encoded variable to a new column using the cat.nodes accessor:

df["WEATHER_CAT"] = df["WEATHER"].cat.codes
df.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND,WEATHER_CAT
0,2,Overcast,Wet,Daylight,4
1,1,Raining,Wet,Dark - Street Lights On,6
2,1,Overcast,Dry,Daylight,4
3,1,Clear,Dry,Daylight,1
4,2,Raining,Wet,Daylight,6


In [55]:
#Similary, fro LIGHTCOND AND ROADCOND:

df["ROADCOND"] = df["ROADCOND"].astype('category')
df["ROADCOND_CAT"] = df["ROADCOND"].cat.codes


df["LIGHTCOND"] = df["LIGHTCOND"].astype('category')
df["LIGHTCOND_CAT"] = df["LIGHTCOND"].cat.codes

print(df.head())

#Check the datatypes now:

df.dtypes



   SEVERITYCODE   WEATHER ROADCOND                LIGHTCOND  WEATHER_CAT  \
0             2  Overcast      Wet                 Daylight            4   
1             1   Raining      Wet  Dark - Street Lights On            6   
2             1  Overcast      Dry                 Daylight            4   
3             1     Clear      Dry                 Daylight            1   
4             2   Raining      Wet                 Daylight            6   

   ROADCOND_CAT  LIGHTCOND_CAT  
0             8              5  
1             8              2  
2             0              5  
3             0              5  
4             8              5  


SEVERITYCODE        int64
WEATHER          category
ROADCOND         category
LIGHTCOND        category
WEATHER_CAT          int8
ROADCOND_CAT         int8
LIGHTCOND_CAT        int8
dtype: object

In [56]:
df.describe()

,SEVERITYCODE,WEATHER_CAT,ROADCOND_CAT,LIGHTCOND_CAT
count,194673.000000,194673.000000,194673.000000,194673.000000
mean,1.298901,2.977254,2.507122,4.256420
std,0.457778,2.892011,3.648660,1.900722
min,1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,1.000000,0.000000,2.000000
50%,1.000000,1.000000,0.000000,5.000000
75%,2.000000,6.000000,7.000000,5.000000
max,2.000000,10.000000,8.000000,8.000000


In [57]:
df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [60]:
# As we can see above, for Severity code for class 1 is nearly 3 times to that of class 2
# Hence we have to balance our dataset ie the "Severitycode" column i.e. our target varibale.
# So any of the balancing techniques can be used, here we will be using down sampling (Sampling down class 1)

#Separate majority and minority classes
df_majority = df[df.SEVERITYCODE==1]
df_minority = df[df.SEVERITYCODE==2]

#downsample the majority class
df_majority_downsampled = resample(df_majority,replace=False,n_samples=58188,random_state=123)

#Combine the minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

df_downsampled['SEVERITYCODE'].value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [70]:
# Initialize, Normalize and Train/test split:

# 1) Initialize the dataset with dependent and independent variables respectively:
X = np.asarray(df_downsampled[['WEATHER_CAT', 'LIGHTCOND_CAT', 'ROADCOND_CAT']])
print("Initilazied X array with independent variables \n" ,X[0:5])

Y= np.asarray(df_downsampled['SEVERITYCODE'])
print("Initialized y array with target variable \n", Y[0:5])

# 2) Normalize the dataset
X = preprocessing.StandardScaler().fit(X).transform(X)
print("\n Normalized data set \n", X[0:5])

# 3) Train/Test split - We will use 30% of data for testing and 70% of data for training:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=4)
print('\n Train set:',X_train.shape,y_train.shape)
print('\n Test set:',X_test.shape,y_test.shape)

Initilazied X array with independent variables 
 [[ 6  2  8]
 [ 1  5  0]
 [10  8  7]
 [ 1  5  0]
 [ 1  5  0]]
Initialized y array with target variable 
 [1 1 1 1 1]

 Normalized data set 
 [[ 1.15236718 -1.21648407  1.52797946]
 [-0.67488     0.42978835 -0.67084969]
 [ 2.61416492  2.07606076  1.25312582]
 [-0.67488     0.42978835 -0.67084969]
 [-0.67488     0.42978835 -0.67084969]]

 Train set: (81463, 3) (81463,)

 Test set: (34913, 3) (34913,)


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [80]:
# Use KNN:
from sklearn.neighbors import KNeighborsClassifier

#Building the KNN model:
k=25
neigh = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
neigh

#predicting using test data:
yhat = neigh.predict(X_test)
yhat[0:5]

array([2, 2, 1, 1, 2])

In [85]:
#Metrics and Evaluation for KNN:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_similarity_score

print("Jaccard similarity score for evaluation:\t", jaccard_similarity_score(y_test, yhat))

print("F1 score for evaluation:\t", f1_score(y_test, yhat, average='macro'))


Jaccard similarity score for evaluation:	 0.564001947698565
F1 score for evaluation:	 0.5401775308974308


In [91]:
# Decision Tree:
from sklearn.tree import DecisionTreeClassifier

#Building the Decision tree
Tree = DecisionTreeClassifier(criterion="entropy",max_depth=6)
Tree
Tree.fit(X_train,y_train)

#Train and predict
Dtyhat = Tree.predict(X_test)
print("Predicted output:\n", Dtyhat[0:5])
print("The original Test set:\n" ,y_test[0:5])


Predicted output:
 [2 2 1 1 2]
The original Test set:
 [2 2 1 1 1]


In [92]:
#Metrics and Evaluation for Decision Tree:

print("Jaccard similarity score for evaluation:\t", jaccard_similarity_score(y_test, Dtyhat))

print("F1 score for evaluation:\t", f1_score(y_test, Dtyhat, average='macro'))


Jaccard similarity score for evaluation:	 0.5647180133474637
F1 score for evaluation:	 0.5423044036658331


In [95]:
# Logistic Regression:
from sklearn.linear_model import LogisticRegression
LogR = LogisticRegression(C=6, solver='liblinear').fit(X_train,y_train)
LogR

# Train and predict the model:
LRyhat = LogR.predict(X_test)
LRyhat

array([1, 2, 1, ..., 2, 2, 2])

In [96]:
yhat_prob = LogR.predict_proba(X_test)
yhat_prob

array([[0.57295252, 0.42704748],
       [0.47065071, 0.52934929],
       [0.67630201, 0.32369799],
       ...,
       [0.46929132, 0.53070868],
       [0.47065071, 0.52934929],
       [0.46929132, 0.53070868]])

In [98]:
# Metrics and Evaluation:

print("Jaccard similarity score for evaluation:\t", jaccard_similarity_score(y_test, LRyhat))

print("F1 score for evaluation:\t", f1_score(y_test, LRyhat, average='macro'))

# Logloss evaluation: 
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

Jaccard similarity score for evaluation:	 0.5260218256809784
F1 score for evaluation:	 0.511602093963383


0.6849535383198887